In [ ]:
#==========================================================
# STEP 1: Install required libraries
#==========================================================
!pip install transformers datasets torch scikit-learn matplotlib

import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#==========================================================
# STEP 2: Load your dataset (Upload CSV to Colab first)
#==========================================================
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0], encoding='latin1')

#🔹 Replace column names here
TEXT_COL = "text"     # your text column name
LABEL_COL = "sentiment"   # label column name

df = df[[TEXT_COL, LABEL_COL]]
print(df.head())

In [ ]:
#==========================================================
# STEP 3: Preprocessing
#==========================================================
df.dropna(inplace=True)

# Map string labels to integers
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
df[LABEL_COL] = df[LABEL_COL].map(label_mapping)
df[LABEL_COL] = df[LABEL_COL].astype(int)  # ensure numeric labels

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#==========================================================
# STEP 4: Tokenization
#==========================================================
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch[TEXT_COL], padding=True, truncation=True, max_length=256)

train_encodings = tokenizer(list(train_df[TEXT_COL]), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df[TEXT_COL]), truncation=True, padding=True)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, list(train_df[LABEL_COL]))
test_dataset  = Dataset(test_encodings , list(test_df[LABEL_COL]))


In [ ]:
#==========================================================
# STEP 5: Model Setup
#==========================================================
num_labels = df[LABEL_COL].nunique()
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

In [ ]:
#==========================================================
# STEP 6: Training Arguments
#==========================================================
EPOCHS = 5  # 🔹 Change epochs here manually or ask user input

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
#==========================================================
# STEP 7: Metrics Function
#==========================================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
#==========================================================
# STEP 8: Train Model
#==========================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

In [ ]:
#==========================================================
# STEP 9: Confusion Matrix (Train + Test)
#==========================================================
def plot_confusion(dataset, title):
    preds = trainer.predict(dataset)
    cm = confusion_matrix(preds.label_ids, preds.predictions.argmax(-1))
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

print("\n📊 TRAIN CONFUSION MATRIX")
plot_confusion(train_dataset,"Train Confusion Matrix")

print("\n📊 TEST CONFUSION MATRIX")
plot_confusion(test_dataset,"Test Confusion Matrix")

In [ ]:
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

def predict_text(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Move tokens to the same device as the model
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    output = model(**tokens)
    pred = torch.argmax(output.logits).item()
    return label_map[pred]
predict_text("This laptop works very smoothly, I love it!")